In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import optuna
import optuna.visualization as vis
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf

In [2]:
testPath = r'C:\Users\LENOVO\OneDrive\Desktop\Python\Datasets\isis\test-metadata.csv'
trainPath = r'C:\Users\LENOVO\OneDrive\Desktop\Python\Datasets\isis\train-metadata.csv'

In [3]:
testData = pd.read_csv(testPath)
trainData = pd.read_csv(trainPath)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_34344\1749218175.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  trainData = pd.read_csv(trainPath)


In [4]:
trainColumns = set(trainData.columns)
testColumns = set(testData.columns)

In [5]:
trainColumns

{'age_approx',
 'anatom_site_general',
 'attribution',
 'clin_size_long_diam_mm',
 'copyright_license',
 'iddx_1',
 'iddx_2',
 'iddx_3',
 'iddx_4',
 'iddx_5',
 'iddx_full',
 'image_type',
 'isic_id',
 'lesion_id',
 'mel_mitotic_index',
 'mel_thick_mm',
 'patient_id',
 'sex',
 'target',
 'tbp_lv_A',
 'tbp_lv_Aext',
 'tbp_lv_B',
 'tbp_lv_Bext',
 'tbp_lv_C',
 'tbp_lv_Cext',
 'tbp_lv_H',
 'tbp_lv_Hext',
 'tbp_lv_L',
 'tbp_lv_Lext',
 'tbp_lv_areaMM2',
 'tbp_lv_area_perim_ratio',
 'tbp_lv_color_std_mean',
 'tbp_lv_deltaA',
 'tbp_lv_deltaB',
 'tbp_lv_deltaL',
 'tbp_lv_deltaLB',
 'tbp_lv_deltaLBnorm',
 'tbp_lv_dnn_lesion_confidence',
 'tbp_lv_eccentricity',
 'tbp_lv_location',
 'tbp_lv_location_simple',
 'tbp_lv_minorAxisMM',
 'tbp_lv_nevi_confidence',
 'tbp_lv_norm_border',
 'tbp_lv_norm_color',
 'tbp_lv_perimeterMM',
 'tbp_lv_radial_color_std_max',
 'tbp_lv_stdL',
 'tbp_lv_stdLExt',
 'tbp_lv_symm_2axis',
 'tbp_lv_symm_2axis_angle',
 'tbp_lv_x',
 'tbp_lv_y',
 'tbp_lv_z',
 'tbp_tile_type'}

In [6]:
testColumns

{'age_approx',
 'anatom_site_general',
 'attribution',
 'clin_size_long_diam_mm',
 'copyright_license',
 'image_type',
 'isic_id',
 'patient_id',
 'sex',
 'tbp_lv_A',
 'tbp_lv_Aext',
 'tbp_lv_B',
 'tbp_lv_Bext',
 'tbp_lv_C',
 'tbp_lv_Cext',
 'tbp_lv_H',
 'tbp_lv_Hext',
 'tbp_lv_L',
 'tbp_lv_Lext',
 'tbp_lv_areaMM2',
 'tbp_lv_area_perim_ratio',
 'tbp_lv_color_std_mean',
 'tbp_lv_deltaA',
 'tbp_lv_deltaB',
 'tbp_lv_deltaL',
 'tbp_lv_deltaLB',
 'tbp_lv_deltaLBnorm',
 'tbp_lv_eccentricity',
 'tbp_lv_location',
 'tbp_lv_location_simple',
 'tbp_lv_minorAxisMM',
 'tbp_lv_nevi_confidence',
 'tbp_lv_norm_border',
 'tbp_lv_norm_color',
 'tbp_lv_perimeterMM',
 'tbp_lv_radial_color_std_max',
 'tbp_lv_stdL',
 'tbp_lv_stdLExt',
 'tbp_lv_symm_2axis',
 'tbp_lv_symm_2axis_angle',
 'tbp_lv_x',
 'tbp_lv_y',
 'tbp_lv_z',
 'tbp_tile_type'}

In [7]:
ColumnsOnlyInTrain = list(trainColumns.difference(testColumns))
ColumnsOnlyInTrain.remove('target')
ColumnsOnlyInTrain

['mel_thick_mm',
 'lesion_id',
 'iddx_full',
 'iddx_2',
 'mel_mitotic_index',
 'iddx_1',
 'iddx_5',
 'tbp_lv_dnn_lesion_confidence',
 'iddx_3',
 'iddx_4']

In [8]:
trainCopy = trainData.copy()

In [9]:
trainCopy.drop(columns=ColumnsOnlyInTrain, inplace=True)

In [10]:
trainCopy

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,0.000000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,0.000000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,Memorial Sloan Kettering Cancer Center,CC-BY
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,0.000000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,Memorial Sloan Kettering Cancer Center,CC-BY
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,0.664690,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,ACEMID MIA,CC-0
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,0.000000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,Memorial Sloan Kettering Cancer Center,CC-BY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,0,IP_1140263,70.0,male,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,...,2.328066,7.054819,2.169398,0.288920,100,147.187256,1182.317505,122.652588,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC
401055,ISIC_9999951,0,IP_5678181,60.0,male,posterior torso,3.11,TBP tile: close-up,3D: white,19.977640,...,1.048147,1.879502,2.910780,0.460000,25,52.349740,1393.187000,127.261700,Memorial Sloan Kettering Cancer Center,CC-BY
401056,ISIC_9999960,0,IP_0076153,65.0,female,anterior torso,2.05,TBP tile: close-up,3D: XP,17.332567,...,0.607554,1.702824,2.205272,0.183099,40,54.622246,1065.263672,-106.833740,"Frazer Institute, The University of Queensland...",CC-BY
401057,ISIC_9999964,0,IP_5231513,30.0,female,anterior torso,2.80,TBP tile: close-up,3D: XP,22.288570,...,1.078204,3.680175,1.957157,0.161850,140,-9.861557,877.527000,-76.982120,University Hospital of Basel,CC-BY-NC


In [11]:
size = trainCopy.shape[0]
size

401059

In [12]:
trainCopy.isnull().sum()

isic_id                            0
target                             0
patient_id                         0
age_approx                      2798
sex                            11517
anatom_site_general             5756
clin_size_long_diam_mm             0
image_type                         0
tbp_tile_type                      0
tbp_lv_A                           0
tbp_lv_Aext                        0
tbp_lv_B                           0
tbp_lv_Bext                        0
tbp_lv_C                           0
tbp_lv_Cext                        0
tbp_lv_H                           0
tbp_lv_Hext                        0
tbp_lv_L                           0
tbp_lv_Lext                        0
tbp_lv_areaMM2                     0
tbp_lv_area_perim_ratio            0
tbp_lv_color_std_mean              0
tbp_lv_deltaA                      0
tbp_lv_deltaB                      0
tbp_lv_deltaL                      0
tbp_lv_deltaLB                     0
tbp_lv_deltaLBnorm                 0
t

In [13]:
columnsDrop = ['isic_id', 'patient_id', 'attribution', 'copyright_license', 'image_type']
trainCopy.drop(columns=columnsDrop, inplace=True)
testCopy = testData.copy()
testCopy.drop(columns=columnsDrop, inplace=True)

In [14]:
trainCopy

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z
0,0,60.0,male,lower extremity,3.04,3D: white,20.244422,16.261975,26.922447,23.954773,...,0.000000,9.307003,0.000000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948
1,0,60.0,male,head/neck,1.10,3D: white,31.712570,25.364740,26.331000,24.549290,...,0.000000,3.354148,0.000000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500
2,0,60.0,male,posterior torso,3.40,3D: XP,22.575830,17.128170,37.970460,33.485410,...,0.000000,8.886309,0.000000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900
3,0,65.0,male,anterior torso,3.22,3D: XP,14.242329,12.164757,21.448144,21.121356,...,1.771705,9.514499,0.664690,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802
4,0,55.0,male,anterior torso,2.73,3D: white,24.725520,20.057470,26.464900,25.710460,...,0.000000,6.467562,0.000000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,0,70.0,male,anterior torso,6.80,3D: XP,22.574335,14.944666,27.663259,26.767135,...,7.348126,20.210836,2.328066,7.054819,2.169398,0.288920,100,147.187256,1182.317505,122.652588
401055,0,60.0,male,posterior torso,3.11,3D: white,19.977640,16.026870,34.158840,31.011870,...,2.795246,9.340242,1.048147,1.879502,2.910780,0.460000,25,52.349740,1393.187000,127.261700
401056,0,65.0,female,anterior torso,2.05,3D: XP,17.332567,12.364397,29.845326,26.500073,...,1.660411,5.999862,0.607554,1.702824,2.205272,0.183099,40,54.622246,1065.263672,-106.833740
401057,0,30.0,female,anterior torso,2.80,3D: XP,22.288570,9.564721,28.431200,27.012250,...,3.583966,9.113276,1.078204,3.680175,1.957157,0.161850,140,-9.861557,877.527000,-76.982120


In [15]:
testCopy

,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z
0,45.0,male,posterior torso,2.70,3D: XP,22.80433,20.007270,28.38412,27.043640,36.410100,...,1.143374,9.387248,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100
1,35.0,female,lower extremity,2.52,3D: XP,16.64867,9.657964,31.31752,27.524318,35.467806,...,0.000000,6.340311,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287
2,65.0,male,posterior torso,3.16,3D: XP,24.25384,19.937380,30.46368,28.384240,38.939500,...,0.721739,8.130868,0.230742,1.080308,2.705857,0.366071,110,-84.29282,1303.978000,-28.576050


## Balanced Dataset

In [16]:
X = trainCopy.drop("target", axis=1)  
y = trainCopy["target"]    

In [17]:
ros = RandomOverSampler(random_state=12)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [18]:
y_resampled.value_counts()

target
0    400666
1    400666
Name: count, dtype: int64

In [19]:
X_resampled.shape

(801332, 39)

In [20]:
numericalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categoricalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

categorical_columns = X_resampled.select_dtypes(include=['object','category']).columns
numerical_columns = X_resampled.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(transformers=[
    ('num', numericalTransformer, numerical_columns),
    ('categorical', categoricalTransformer, categorical_columns)
], remainder='passthrough', sparse_threshold=0.0)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [21]:
numerical_columns

Index(['age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext',
       'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H',
       'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2',
       'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA',
       'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
       'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM',
       'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
       'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
       'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
       'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z'],
      dtype='object')

In [22]:
x = pipeline.fit_transform(X_resampled)

In [23]:
test_data = pipeline.transform(testData)

In [24]:
x.shape

(801332, 72)

In [25]:
x_train, x_val, y_train, y_val = train_test_split(x, y_resampled, test_size=0.2, random_state=42)

## **Baseline**

In [41]:
lr = LogisticRegression(max_iter=1000)

lr.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [42]:
val_predictions_lr = lr.predict(x_val)

print('Accuracy:', accuracy_score(y_val, val_predictions_lr))
print('Recall:', recall_score(y_val, val_predictions_lr))
print('Precision:', precision_score(y_val, val_predictions_lr))
print('ROC_AUC',roc_auc_score(y_val, val_predictions_lr))

Accuracy: 0.840091846730768
Recall: 0.8249834748880658
Precision: 0.8508547393334448
ROC_AUC 0.8401008076935147


## **XGBoost**

In [28]:
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',
        'tree_method': 'hist', 
        'device': 'cuda', 
        'predictor': 'gpu_predictor',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 5e-1, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
    }

    model = XGBClassifier(**param)
    model.fit(x_train, y_train)

    preds = model.predict(x_val)
    return recall_score(y_val, preds, average='binary')


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)

[I 2025-05-08 00:31:42,398] A new study created in memory with name: no-name-dc9414de-481a-4e7e-912b-7b76a4328dc8
[I 2025-05-08 00:31:46,574] Trial 0 finished with value: 0.9693817737369202 and parameters: {'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.006207952248867322, 'colsample_bytree': 0.788094272478499, 'reg_alpha': 0.9246110238175279, 'reg_lambda': 0.6345759178016177}. Best is trial 0 with value: 0.9693817737369202.
[I 2025-05-08 00:31:53,871] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 379, 'max_depth': 12, 'learning_rate': 0.1551825862531018, 'colsample_bytree': 0.7381510595414473, 'reg_alpha': 0.6812914666932334, 'reg_lambda': 0.1569547843554635}. Best is trial 1 with value: 1.0.
[I 2025-05-08 00:32:06,646] Trial 2 finished with value: 0.8444269839488158 and parameters: {'n_estimators': 990, 'max_depth': 4, 'learning_rate': 0.00010221102479331568, 'colsample_bytree': 0.7452451185686357, 'reg_alpha': 0.21431332843583462, 'reg_lambda': 0.9556242

Best params: {'n_estimators': 379, 'max_depth': 12, 'learning_rate': 0.1551825862531018, 'colsample_bytree': 0.7381510595414473, 'reg_alpha': 0.6812914666932334, 'reg_lambda': 0.1569547843554635}
Best RMSE: 1.0


In [29]:
vis.plot_optimization_history(study).show()

In [30]:
vis.plot_param_importances(study).show()

In [31]:
vis.plot_parallel_coordinate(study).show()

In [32]:
params = {'objective': 'binary:logistic', 
          'tree_method': 'hist', 
          'device': 'cuda',
          'n_estimators': 379, 
          'max_depth': 12, 
          'learning_rate': 0.1551825862531018, 
          'colsample_bytree': 0.7381510595414473, 
          'reg_alpha': 0.6812914666932334, 
          'reg_lambda': 0.1569547843554635}

xgb = XGBClassifier(**params)
xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7381510595414473, device='cuda',
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1551825862531018,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=379, n_jobs=None,
              num_parallel_tree=None, ...)

In [33]:
val_predictions_xgb = xgb.predict(x_val)

print('Accuracy:', accuracy_score(y_val, val_predictions_xgb))
print('Recall:', recall_score(y_val, val_predictions_xgb))
print('Precision:', precision_score(y_val, val_predictions_xgb))
print('ROC_AUC',roc_auc_score(y_val, val_predictions_xgb))

Accuracy: 0.999856489483175
Recall: 1.0
Precision: 0.9997132312602862
ROC_AUC 0.9998564043653073


In [34]:
predictions_xgb = xgb.predict_proba(test_data)
predictions_xgb

array([[9.9999750e-01, 2.5083243e-06],
       [9.9999577e-01, 4.2093639e-06],
       [9.9997830e-01, 2.1691958e-05]], dtype=float32)

## **LGBM**

In [35]:
def objective(trial):
    param = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'device': 'gpu', 
        'verbosity': -1,
        'max_bin':255,
        'learning_rate': trial.suggest_float('learning_rate', 5e-3, 5e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 16, 1024),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 250),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
    }
    
    model = LGBMClassifier(**param)
        
    model.fit(np.array(x_train), np.array(y_train))

    y_pred = model.predict(np.array(x_val))
    return recall_score(y_val, y_pred, average='binary')
     

study = optuna.create_study(direction="maximize")  
study.optimize(objective, n_trials=10)  

print("Best params:", study.best_params)
print("Best Recall:", study.best_value)


[I 2025-05-08 00:40:49,543] A new study created in memory with name: no-name-f11b8e5c-0c30-4b48-bbb4-bdfbe0800bd2
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names

[I 2025-05-08 00:41:37,514] Trial 0 finished with value: 1.0 and parameters: {'learning_rate': 0.009646282271546798, 'num_leaves': 981, 'max_depth': 7, 'min_child_samples': 18, 'colsample_bytree': 0.7630572019109578, 'reg_alpha': 2.501382048184091, 'reg_lambda': 0.0003483839539085421, 'n_estimators': 1287}. Best is trial 0 with value: 1.0.
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names

[I 2025-05-08 00:42:12,327] Trial 1 finished with value: 1.0 and parameters: {'learning_rate': 0.11033743485992598, 'num_leave

Best params: {'learning_rate': 0.009646282271546798, 'num_leaves': 981, 'max_depth': 7, 'min_child_samples': 18, 'colsample_bytree': 0.7630572019109578, 'reg_alpha': 2.501382048184091, 'reg_lambda': 0.0003483839539085421, 'n_estimators': 1287}
Best Recall: 1.0


In [37]:
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_parallel_coordinate(study).show()

In [38]:
params = {'objective': 'binary',
          'boosting_type': 'gbdt',
          'device': 'gpu', 
          'verbosity': -1,
          'max_bin':255,
          'learning_rate': 0.009646282271546798, 
          'num_leaves': 981, 
          'max_depth': 7, 
          'min_child_samples': 18, 
          'colsample_bytree': 0.7630572019109578, 
          'reg_alpha': 2.501382048184091, 
          'reg_lambda': 0.0003483839539085421, 
          'n_estimators': 1287}


lgb = LGBMClassifier(**params)
lgb.fit(x_train, y_train)

LGBMClassifier(colsample_bytree=0.7630572019109578, device='gpu',
               learning_rate=0.009646282271546798, max_bin=255, max_depth=7,
               min_child_samples=18, n_estimators=1287, num_leaves=981,
               objective='binary', reg_alpha=2.501382048184091,
               reg_lambda=0.0003483839539085421, verbosity=-1)

In [39]:
val_predictions_lgb = lgb.predict(x_val)

print('Accuracy:', accuracy_score(y_val, val_predictions_lgb))
print('Recall:', recall_score(y_val, val_predictions_lgb))
print('Precision:', precision_score(y_val, val_predictions_lgb))
print('ROC_AUC',roc_auc_score(y_val, val_predictions_lgb))

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names



Accuracy: 0.9978473422476243
Recall: 1.0
Precision: 0.9957156694732137
ROC_AUC 0.9978460654796094


In [40]:
predictions_lgb = lgb.predict_proba(test_data)
predictions_lgb

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names



array([[9.99448023e-01, 5.51977297e-04],
       [9.99898189e-01, 1.01810956e-04],
       [9.93792550e-01, 6.20744958e-03]])

In [44]:
def create_model(trial):
    model = tf.keras.models.Sequential()
    
    num_layers = trial.suggest_int("num_layers", 1, 5)
    
    for i in range(num_layers):
        num_units = trial.suggest_int(f"num_units_{i}", 64, 256)
        model.add(tf.keras.layers.Dense(num_units, activation="relu"))
    
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    model.add(tf.keras.layers.Dropout(dropout_rate))
    
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)),
        metrics=["accuracy", "recall"]
    )
    
    return model

def objective(trial):
    model = create_model(trial)
    
    history = model.fit(x_train, y_train, 
                        epochs=10, 
                        batch_size=trial.suggest_int("batch_size", 32, 512),
                        validation_data=(x_val, y_val),
                        verbose=0)
    
    return history.history["val_recall"][-1]

study = optuna.create_study(direction="maximize")  
study.optimize(objective, n_trials=10)

print("Best hyperparameters:", study.best_params)

[I 2025-05-08 00:58:59,082] A new study created in memory with name: no-name-fa84d9c2-062f-4d48-a326-5169fb2f3a8a
[I 2025-05-08 01:00:29,752] Trial 0 finished with value: 1.0 and parameters: {'num_layers': 5, 'num_units_0': 104, 'num_units_1': 204, 'num_units_2': 193, 'num_units_3': 170, 'num_units_4': 230, 'dropout_rate': 0.4256062425873889, 'learning_rate': 0.00011025607615807657, 'batch_size': 339}. Best is trial 0 with value: 1.0.
[I 2025-05-08 01:01:48,829] Trial 1 finished with value: 1.0 and parameters: {'num_layers': 5, 'num_units_0': 170, 'num_units_1': 95, 'num_units_2': 153, 'num_units_3': 179, 'num_units_4': 142, 'dropout_rate': 0.24835772783539814, 'learning_rate': 0.00068870185565164, 'batch_size': 317}. Best is trial 0 with value: 1.0.
[I 2025-05-08 01:03:01,014] Trial 2 finished with value: 1.0 and parameters: {'num_layers': 3, 'num_units_0': 155, 'num_units_1': 134, 'num_units_2': 112, 'dropout_rate': 0.24664550920262754, 'learning_rate': 0.009448766511165175, 'batch_s

Best hyperparameters: {'num_layers': 5, 'num_units_0': 104, 'num_units_1': 204, 'num_units_2': 193, 'num_units_3': 170, 'num_units_4': 230, 'dropout_rate': 0.4256062425873889, 'learning_rate': 0.00011025607615807657, 'batch_size': 339}


In [45]:
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_parallel_coordinate(study).show()

In [46]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(104, activation='relu'),
    tf.keras.layers.Dense(204, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(193, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(170, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(230, activation='relu'),
    tf.keras.layers.Dropout(0.4256062425873889),
    tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00011025607615807657),
              metrics=["recall", "accuracy", "precision"])

In [48]:
model.fit(x_train,
          y_train,
          validation_data=(x_val, y_val),
          epochs=10,
          batch_size=512)

Epoch 1/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7788 - loss: 0.7610 - precision: 0.8739 - recall: 0.6301 - val_accuracy: 0.9656 - val_loss: 0.3231 - val_precision: 0.9402 - val_recall: 0.9945
Epoch 2/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9774 - loss: 0.2831 - precision: 0.9635 - recall: 0.9924 - val_accuracy: 0.9882 - val_loss: 0.2222 - val_precision: 0.9770 - val_recall: 1.0000
Epoch 3/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9899 - loss: 0.2131 - precision: 0.9809 - recall: 0.9991 - val_accuracy: 0.9907 - val_loss: 0.2006 - val_precision: 0.9913 - val_recall: 0.9902
Epoch 4/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9927 - loss: 0.1815 - precision: 0.9866 - recall: 0.9988 - val_accuracy: 0.9862 - val_loss: 0.1721 - val_precision: 0.9910 - val_recall: 0.9813
Epoch 5/10
1253/1253 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9937 - loss: 0.1622 - precision: 0.9887 - recall: 0.9988 - val_accuracy: 0.9950 - v

In [51]:
predictions_nn = model.predict(test_data)
predictions_nn

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[-18.917912],
       [-56.400898],
       [-17.129515]], dtype=float32)

In [52]:
predictions_nn = 1/(1 + np.exp(-predictions_nn))
predictions_nn

array([[6.0821259e-09],
       [3.2018518e-25],
       [3.6370249e-08]], dtype=float32)

In [55]:
model.save('DataframeModel.keras')
xgb.save_model('DataframeXGB.json')